Created a data with python code to keep project simple and easy to explore

In [1]:
import pandas as pd
import random
import numpy as np
from datetime import datetime, timedelta


In [2]:
# import pandas as pd
# import random
# import numpy as np
# from datetime import datetime, timedelta

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Sample customers
customers = [f"Customer_{i}" for i in range(1, 11)]

# Define asset types and other variables
single_asset_types = ["Stocks", "Gold", "Real Estate", "Bonds"]
multi_asset_types = [
    ["Gold", "Stocks", "Bonds"],
    ["Stocks", "Real Estate", "Bonds"],
    ["Bonds", "Gold", "Stocks"],
    ["Bonds", "Stocks", "Crypto"],
    ["Gold", "Crypto"],
    ["Stocks", "Real Estate"]
]
# investment_statuses = ["Profitable", "At Risk", "Stable", "Loss"]
# risk_levels = ["Low", "Medium", "High"]

# Helper function to generate a random date in 2022
def random_date():
    start_date = datetime(2022, 1, 1)
    end_date = datetime(2022, 12, 31)
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    return (start_date + timedelta(days=random_days)).strftime("%Y-%m-%d")

# Build dataset
data = []
for i, cust in enumerate(customers):
    # Ensure 6 of 10 customers have multi-asset investments
    if i < 6:
        assets = random.choice(multi_asset_types)
    else:
        assets = [random.choice(single_asset_types)]
    
    # Generate values for each asset
    inv_amounts = [round(random.uniform(5000, 50000), 2) for _ in assets]
    # curr_values = [round(random.uniform(5000, 60000), 2) for _ in assets]
    # rois = [round(random.uniform(-10, 20), 2) for _ in assets]
    
    # Generate random dates for each asset investment
    inv_dates = [random_date() for _ in assets]
    
    # For simplicity, choose a single overall risk level and status per customer
    # risk_level = random.choice(risk_levels)
    # status = random.choice(investment_statuses)
    
    # Append data: Join multi-asset details as comma separated strings
    data.append({
        "Customer_ID": cust,
        "Asset_Types": ", ".join(assets),
        "Investment_Amounts": ", ".join(map(str, inv_amounts)),
        "Current_Values": ", ".join(map(str, curr_values)),
        #"ROI (%)": ", ".join(map(str, rois)),
        # "Risk_Level": risk_level,
        # "Investment_Status": status,
        "Investment_Date": ", ".join(inv_dates)
    })

# Create DataFrame
df = pd.DataFrame(data)
print(df)


   Customer_ID            Asset_Types            Investment_Amounts  \
0   Customer_1    Stocks, Real Estate             10009.9, 38369.77   
1   Customer_2           Gold, Crypto             23986.48, 6340.87   
2   Customer_3           Gold, Crypto            23878.39, 25214.41   
3   Customer_4    Bonds, Gold, Stocks  11996.58, 48074.59, 20146.75   
4   Customer_5    Gold, Stocks, Bonds  29841.83, 42323.21, 32833.39   
5   Customer_6  Bonds, Stocks, Crypto   17508.81, 33605.8, 21417.45   
6   Customer_7                   Gold                      25801.71   
7   Customer_8                   Gold                      35807.64   
8   Customer_9                 Stocks                      41227.06   
9  Customer_10            Real Estate                      14568.19   

                 Current_Values              ROI (%)  \
0            18469.05, 12674.59         -6.93, 12.22   
1            17025.09, 32794.54          -9.2, -4.03   
2            20300.49, 52811.52         12.76, -5.

Explanation:
Customer_ID: Unique identifier for each customer.
Asset_Types: A comma-separated list of assets the customer has invested in.
Investment_Amounts: Corresponding investment amounts.
Current_Values: Current value of each asset.
ROI (%): Return on investment for each asset.
Risk_Level & Investment_Status: Assigned overall per customer.
Investment_Date: Comma-separated list of dates when the investments were made.
This dataset meets the requirement that 6 of the 10 customers have invested in more than 2 different types of assets, with each asset including an investment date. Let me know if you need further adjustments!

2. Dynamic Data (Fetched via API)
To update the current asset value, we integrate APIs such as:

Stock Market API (Yahoo Finance, Alpha Vantage, IEX Cloud) → for real-time stock prices
Cryptocurrency API (CoinGecko, Binance API) → for live crypto prices
Gold API (Metals-API) → for live gold rates
Real Estate API (Zillow, Realtor API) → for property valuations
3. API Integration


In [9]:
import requests
import pandas as pd
import random
import numpy as np
from datetime import datetime, timedelta

# Alpha Vantage API Key (Get it from https://www.alphavantage.co/)
API_KEY = "your_alpha_vantage_api_key"

# API Base URL
BASE_URL = "https://www.alphavantage.co/query"

# List of sample stock symbols (for real API fetching)
stock_symbols = ["AAPL", "GOOGL", "AMZN", "MSFT", "TSLA", "META", "NVDA", "JPM", "V", "BRK.B"]

# Define asset types (can be expanded)
asset_types = ["Stocks", "Gold", "Real Estate", "Bonds", "Crypto"]
multi_asset_types = [
    ["Gold", "Stocks", "Bonds"],
    ["Stocks", "Real Estate", "Bonds"],
    ["Bonds", "Gold", "Stocks"],
    ["Bonds", "Stocks", "Crypto"],
    ["Gold", "Crypto"],
    ["Stocks", "Real Estate"]
]

# investment_statuses = ["Profitable", "At Risk", "Stable", "Loss"]
# risk_levels = ["Low", "Medium", "High"]

# Helper function to generate a random date in 2022
def random_date():
    start_date = datetime(2020, 1, 1)
    end_date = datetime(2024, 12, 31)
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    return (start_date + timedelta(days=random_days)).strftime("%Y-%m-%d")

# Function to fetch real-time stock prices from Alpha Vantage
def fetch_stock_price(symbol):
    params = {
        "function": "TIME_SERIES_INTRADAY",
        "symbol": symbol,
        "interval": "5min",
        "apikey": API_KEY
    }
    response = requests.get(BASE_URL, params=params)
    data = response.json()
    
    try:
        latest_time = list(data["Time Series (5min)"].keys())[0]
        price = float(data["Time Series (5min)"][latest_time]["1. open"])
        return price
    except KeyError:
        return round(random.uniform(50, 500), 2)  # Fallback to random value

# Function to fetch cryptocurrency price (e.g., Bitcoin)
def fetch_crypto_price(symbol="BTC"):
    params = {
        "function": "CURRENCY_EXCHANGE_RATE",
        "from_currency": symbol,
        "to_currency": "USD",
        "apikey": API_KEY
    }
    response = requests.get(BASE_URL, params=params)
    data = response.json()
    
    try:
        price = float(data["Realtime Currency Exchange Rate"]["5. Exchange Rate"])
        return price
    except KeyError:
        return round(random.uniform(10000, 60000), 2)  # Fallback to random value

# Function to generate investment data
def generate_investment_data():
    data = []
    
    for i in range(10):  # 10 Customers
        cust_id = f"Customer_{i+1}"
        
        # Ensure 6 of 10 customers have multi-asset investments
        if i < 6:
            assets = random.choice(multi_asset_types)
        else:
            assets = [random.choice(asset_types)]
        
        # Generate values for each asset
        inv_amounts = [round(random.uniform(5000, 50000), 2) for _ in assets]
        inv_dates = [random_date() for _ in assets]

        current_values = []
        for asset in assets:
            if asset == "Stocks":
                stock_symbol = random.choice(stock_symbols)
                current_values.append(fetch_stock_price(stock_symbol))
            elif asset == "Crypto":
                current_values.append(fetch_crypto_price())
            else:
                # Gold, Real Estate, Bonds - Simulating values (real API can be integrated here)
                current_values.append(round(random.uniform(5000, 60000), 2))
        
        # rois = [round((cv - ia) / ia * 100, 2) for cv, ia in zip(current_values, inv_amounts)]
        # risk_level = random.choice(risk_levels)
        # status = random.choice(investment_statuses)
        
        # Append data: Join multi-asset details as comma-separated strings
        data.append({
            "Customer_ID": cust_id,
            "Asset_Types": ", ".join(assets),
            "Investment_Amounts": ", ".join(map(str, inv_amounts)),
            "Current_Values": ", ".join(map(str, current_values)),
            # "ROI (%)": ", ".join(map(str, rois)),
            # "Risk_Level": risk_level,
            # "Investment_Status": status,
            "Investment_Date": ", ".join(inv_dates)
        })

    return pd.DataFrame(data)

# Generate DataFrame
df = generate_investment_data()
print(df)


   Customer_ID                 Asset_Types            Investment_Amounts  \
0   Customer_1         Bonds, Gold, Stocks  46693.31, 40330.82, 17836.22   
1   Customer_2  Stocks, Real Estate, Bonds  18351.85, 48591.92, 31063.11   
2   Customer_3                Gold, Crypto            46372.42, 28900.68   
3   Customer_4  Stocks, Real Estate, Bonds    8083.46, 8058.24, 43781.86   
4   Customer_5       Bonds, Stocks, Crypto  34581.79, 30435.21, 19236.64   
5   Customer_6         Bonds, Gold, Stocks  22810.36, 35226.06, 18499.87   
6   Customer_7                      Stocks                      29193.55   
7   Customer_8                 Real Estate                      44638.89   
8   Customer_9                 Real Estate                      12098.61   
9  Customer_10                 Real Estate                       32525.5   

                Current_Values                     Investment_Date  
0   35567.26, 44512.46, 223.29  2023-11-27, 2024-02-05, 2024-05-23  
1    694.05, 8144.09, 371

1. Gold Prices API
Gold prices can be retrieved using financial market APIs that track commodities.

✅ Recommended APIs:
API	Features	        Pricing
Metals-API	Real-time & historical gold, silver, and other metals prices	Free (limited requests), Paid

Alpha Vantage	Gold price in USD (XAU/USD) via CURRENCY_EXCHANGE_RATE	Free (rate-limited)

GoldAPI	Live and historical gold & silver prices	Free & Paid


In [7]:
pip install yahooquery



   ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
   ---------- ----------------------------- 1.0/3.9 MB 7.2 MB/s eta 0:00:01
   ------------------------ --------------- 2.4/3.9 MB 6.1 MB/s eta 0:00:01
   ---------------------------------- ----- 3.4/3.9 MB 5.6 MB/s eta 0:00:01
   ------------------------------------- -- 3.7/3.9 MB 5.1 MB/s eta 0:00:01
   ------------------------------------- -- 3.7/3.9 MB 5.1 MB/s eta 0:00:01
   ------------------------------------- -- 3.7/3.9 MB 5.1 MB/s eta 0:00:01
   ------------------------------------- -- 3.7/3.9 MB 5.1 MB/s eta 0:00:01
   ------------------------------------- -- 3.7/3.9 MB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 3.9/3.9 MB 1.9 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.3.0
    Uninstalling lxml-5.3.0:
      Successfully uninstalled lxml-5.3.0


In [8]:
from yahooquery import Ticker

# Yahoo Finance ticker for Gold Futures
gold = Ticker("GC=F")

# Fetch real-time price
gold_price = gold.price["GC=F"]["regularMarketPrice"]

print(f"Current Gold Price: ${gold_price} per ounce")


Current Gold Price: $2957.6 per ounce


💡 Additional Data from Yahoo Finance
You can also fetch:

Historical prices (gold.history())
Market summary (gold.summary_detail)
Exchange rate trends (gold.price)


2. Real Estate API (Housing Market Data)
Real estate market data APIs provide home prices, rental estimates, and historical trends.

✅ Recommended APIs:
API	Features	Pricing
Zillow API	Home price estimates, rental data, property details	Requires approval
Realtor API	US real estate listings, property values, mortgage rates	Free & Paid
ATTOM Data	US property values, ownership, tax data	Paid


In [ ]:
import requests

api_key = "your_rapidapi_key"
url = "https://realtor.p.rapidapi.com/properties/v2/list-for-sale"
headers = {
    "X-RapidAPI-Key": api_key,
    "X-RapidAPI-Host": "realtor.p.rapidapi.com"
}
params = {"city": "Los Angeles", "state_code": "CA", "limit": "1"}

response = requests.get(url, headers=headers, params=params).json()
property_price = response["properties"][0]["price"]
print(f"Property Price: ${property_price}")


3. Bonds API (Government & Corporate Bonds)
Bond data APIs provide bond yields, interest rates, and treasury rates.

✅ Recommended APIs:
API	Features	Pricing
Alpha Vantage	US Treasury bond yields	Free
Quandl (NASDAQ Data Link)	Bond yields, corporate bond data	Free & Paid
World Government Bonds	Government bond yields	Free


In [ ]:
import requests

api_key = "your_alpha_vantage_api_key"
url = f"https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=10year&apikey={api_key}"

response = requests.get(url).json()
bond_yield = response["data"][0]["value"]
print(f"10-Year Treasury Bond Yield: {bond_yield}%")


In [10]:
print(df)

   Customer_ID                 Asset_Types            Investment_Amounts  \
0   Customer_1         Bonds, Gold, Stocks  46693.31, 40330.82, 17836.22   
1   Customer_2  Stocks, Real Estate, Bonds  18351.85, 48591.92, 31063.11   
2   Customer_3                Gold, Crypto            46372.42, 28900.68   
3   Customer_4  Stocks, Real Estate, Bonds    8083.46, 8058.24, 43781.86   
4   Customer_5       Bonds, Stocks, Crypto  34581.79, 30435.21, 19236.64   
5   Customer_6         Bonds, Gold, Stocks  22810.36, 35226.06, 18499.87   
6   Customer_7                      Stocks                      29193.55   
7   Customer_8                 Real Estate                      44638.89   
8   Customer_9                 Real Estate                      12098.61   
9  Customer_10                 Real Estate                       32525.5   

                Current_Values                     Investment_Date  
0   35567.26, 44512.46, 223.29  2023-11-27, 2024-02-05, 2024-05-23  
1    694.05, 8144.09, 371

In [12]:
import os
localdatapath = os.getcwd()
print(localdatapath)


c:\Jalpa\Data Science UOT\Case Studies\Investment_portfolio


In [13]:
df.to_csv('Cust_Invest_data.csv',index=False)  # save the file in to csv format


In [11]:
# Adding columns 

# ROI
#rois = [round((cv - ia) / ia * 100, 2) for cv, ia in zip(df[Current_Values], inv_amounts)]((

rois= round((df[Current_Values]-df[Investment_Amounts])/df[Investment_Amounts] * 100, 2)

data.append({
            # "Customer_ID": cust_id,
            # "Asset_Types": ", ".join(assets),
            # "Investment_Amounts": ", ".join(map(str, inv_amounts)),
            # "Current_Values": ", ".join(map(str, current_values)),
             "ROI (%)": ", ".join(map(str, rois)),
            # "Risk_Level": risk_level,
            # "Investment_Status": status,
            #"Investment_Date": ", ".join(inv_dates)
        })


NameError: name 'Current_Values' is not defined